## Assemble a corpus

In [1]:
import os; os.getcwd()

'/home/mainuser/Desktop/LLMs/gpt2_from_scratch_to_tiny_stories'

In [2]:
from datasets import load_dataset
tiny_stories_ds=load_dataset("roneneldan/TinyStories")


/home/mainuser/anaconda3/envs/mintonano/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Create Python generator to avoid loading into RAM 

In [3]:
tiny_stories_ds['train'][:5]

{'text': ['One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.',
  'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leav

In [4]:
def get_training_corpus(dataset,split='train',field='text'):
    dataset = dataset[split]
    for start_idx in range(0, len(dataset), 1024): #using a power of 2, why not...
        yield dataset[start_idx:start_idx+1024][field]
training_corpus = get_training_corpus(tiny_stories_ds)

## Train a new tokenizer

- Load a tokenizer primarily for the settings about algorithm, special tokens, etc.

In [5]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained('gpt2')

/home/mainuser/anaconda3/envs/mintonano/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
old_tokenizer#.vocab_size = 50304

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [7]:
old_tokenizer.vocab_size

50257

In [8]:
50256/2/2/2/2 # Mult of 16

3141.0

In [9]:
(2**7)

128

In [10]:
example = tiny_stories_ds['train'][0]['text']

In [11]:
example

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'

In [12]:
tokens_old = old_tokenizer.tokenize(example)

In [13]:
tokens_old

['One',
 'Ġday',
 ',',
 'Ġa',
 'Ġlittle',
 'Ġgirl',
 'Ġnamed',
 'ĠLily',
 'Ġfound',
 'Ġa',
 'Ġneedle',
 'Ġin',
 'Ġher',
 'Ġroom',
 '.',
 'ĠShe',
 'Ġknew',
 'Ġit',
 'Ġwas',
 'Ġdifficult',
 'Ġto',
 'Ġplay',
 'Ġwith',
 'Ġit',
 'Ġbecause',
 'Ġit',
 'Ġwas',
 'Ġsharp',
 '.',
 'ĠLily',
 'Ġwanted',
 'Ġto',
 'Ġshare',
 'Ġthe',
 'Ġneedle',
 'Ġwith',
 'Ġher',
 'Ġmom',
 ',',
 'Ġso',
 'Ġshe',
 'Ġcould',
 'Ġsew',
 'Ġa',
 'Ġbutton',
 'Ġon',
 'Ġher',
 'Ġshirt',
 '.',
 'Ċ',
 'Ċ',
 'L',
 'ily',
 'Ġwent',
 'Ġto',
 'Ġher',
 'Ġmom',
 'Ġand',
 'Ġsaid',
 ',',
 'Ġ"',
 'Mom',
 ',',
 'ĠI',
 'Ġfound',
 'Ġthis',
 'Ġneedle',
 '.',
 'ĠCan',
 'Ġyou',
 'Ġshare',
 'Ġit',
 'Ġwith',
 'Ġme',
 'Ġand',
 'Ġsew',
 'Ġmy',
 'Ġshirt',
 '?"',
 'ĠHer',
 'Ġmom',
 'Ġsmiled',
 'Ġand',
 'Ġsaid',
 ',',
 'Ġ"',
 'Yes',
 ',',
 'ĠLily',
 ',',
 'Ġwe',
 'Ġcan',
 'Ġshare',
 'Ġthe',
 'Ġneedle',
 'Ġand',
 'Ġfix',
 'Ġyour',
 'Ġshirt',
 '."',
 'Ċ',
 'Ċ',
 'Together',
 ',',
 'Ġthey',
 'Ġshared',
 'Ġthe',
 'Ġneedle',
 'Ġand',
 'Ġse',
 'wed',
 'Ġth

In [14]:
len(tokens_old)

162

In [15]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus,vocab_size=6400)#old_tokenizer.vocab_size)#, vocab_size=50304)

In [16]:
tokenizer.vocab_size

12800

In [17]:
old_tokenizer.eos_token

'<|endoftext|>'

In [18]:
tokenizer.eos_token

'<|endoftext|>'

In [19]:
tokens_new = tokenizer.tokenize(example)

In [20]:
tokens_new

['One',
 'Ġday',
 ',',
 'Ġa',
 'Ġlittle',
 'Ġgirl',
 'Ġnamed',
 'ĠLily',
 'Ġfound',
 'Ġa',
 'Ġneedle',
 'Ġin',
 'Ġher',
 'Ġroom',
 '.',
 'ĠShe',
 'Ġknew',
 'Ġit',
 'Ġwas',
 'Ġdifficult',
 'Ġto',
 'Ġplay',
 'Ġwith',
 'Ġit',
 'Ġbecause',
 'Ġit',
 'Ġwas',
 'Ġsharp',
 '.',
 'ĠLily',
 'Ġwanted',
 'Ġto',
 'Ġshare',
 'Ġthe',
 'Ġneedle',
 'Ġwith',
 'Ġher',
 'Ġmom',
 ',',
 'Ġso',
 'Ġshe',
 'Ġcould',
 'Ġsew',
 'Ġa',
 'Ġbutton',
 'Ġon',
 'Ġher',
 'Ġshirt',
 '.',
 'Ċ',
 'Ċ',
 'Lily',
 'Ġwent',
 'Ġto',
 'Ġher',
 'Ġmom',
 'Ġand',
 'Ġsaid',
 ',',
 'Ġ"',
 'Mom',
 ',',
 'ĠI',
 'Ġfound',
 'Ġthis',
 'Ġneedle',
 '.',
 'ĠCan',
 'Ġyou',
 'Ġshare',
 'Ġit',
 'Ġwith',
 'Ġme',
 'Ġand',
 'Ġsew',
 'Ġmy',
 'Ġshirt',
 '?"',
 'ĠHer',
 'Ġmom',
 'Ġsmiled',
 'Ġand',
 'Ġsaid',
 ',',
 'Ġ"',
 'Yes',
 ',',
 'ĠLily',
 ',',
 'Ġwe',
 'Ġcan',
 'Ġshare',
 'Ġthe',
 'Ġneedle',
 'Ġand',
 'Ġfix',
 'Ġyour',
 'Ġshirt',
 '."',
 'Ċ',
 'Ċ',
 'Together',
 ',',
 'Ġthey',
 'Ġshared',
 'Ġthe',
 'Ġneedle',
 'Ġand',
 'Ġsewed',
 'Ġthe',
 'Ġbut

In [21]:
set(tokens_new) & set(tokens_old)

{"'s",
 ',',
 '.',
 '."',
 '?"',
 'Mom',
 'One',
 'Together',
 'Yes',
 'Ċ',
 'Ġ"',
 'ĠAfter',
 'ĠCan',
 'ĠHer',
 'ĠI',
 'ĠIt',
 'ĠLily',
 'ĠShe',
 'ĠThey',
 'Ġa',
 'Ġand',
 'Ġbecause',
 'Ġboth',
 'Ġbutton',
 'Ġcan',
 'Ġcould',
 'Ġday',
 'Ġdifficult',
 'Ġeach',
 'Ġfelt',
 'Ġfinished',
 'Ġfix',
 'Ġfixing',
 'Ġfor',
 'Ġfound',
 'Ġgirl',
 'Ġhad',
 'Ġhappy',
 'Ġhelping',
 'Ġher',
 'Ġin',
 'Ġit',
 'Ġknew',
 'Ġlittle',
 'Ġme',
 'Ġmom',
 'Ġmy',
 'Ġnamed',
 'Ġneedle',
 'Ġnot',
 'Ġon',
 'Ġother',
 'Ġplay',
 'Ġroom',
 'Ġsaid',
 'Ġsew',
 'Ġshare',
 'Ġshared',
 'Ġsharing',
 'Ġsharp',
 'Ġshe',
 'Ġshirt',
 'Ġsmiled',
 'Ġso',
 'Ġthanked',
 'Ġthe',
 'Ġthem',
 'Ġthey',
 'Ġthis',
 'Ġto',
 'Ġtogether',
 'Ġwanted',
 'Ġwas',
 'Ġwe',
 'Ġwent',
 'Ġwere',
 'Ġwith',
 'Ġworked',
 'Ġyou',
 'Ġyour'}

In [22]:
new_minus_old = set(tokens_new) - set(tokens_old)

In [23]:
tokenizer.decode(tokenizer.encode(example))

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'

- Appears to work as expected

In [24]:
set(tokens_old) - set(tokens_new)

{'L', 'ily', 'wed', 'Ġse'}

- Exclusively old tokens are not full French word, at best their closest English counterparts (eg, Person not Personne, Command not Commande, etc)

In [25]:
tokenizer.save_pretrained('./tokenizer/tiny_stories_tokenizer')

('./tokenizer/tiny_stories_tokenizer/tokenizer_config.json',
 './tokenizer/tiny_stories_tokenizer/special_tokens_map.json',
 './tokenizer/tiny_stories_tokenizer/vocab.json',
 './tokenizer/tiny_stories_tokenizer/merges.txt',
 './tokenizer/tiny_stories_tokenizer/added_tokens.json',
 './tokenizer/tiny_stories_tokenizer/tokenizer.json')

In [26]:
tokenizer.encode(tokenizer.eos_token)

[0]

In [27]:
tokenizer.decode([0])

'<|endoftext|>'

In [28]:
tokenizer.encode(' dog'),tokenizer.encode(' girl'), tokenizer.encode(' bird'),tokenizer.encode(' dad'),tokenizer.encode(' mom'),tokenizer.encode(' cat'),tokenizer.encode(' Lily')

([634], [446], [537], [648], [365], [701], [364])

In [29]:
tokenizer.decode([634, 446])

' dog girl'